Importing the neccessary libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
data = pd.read_csv('/content/Credit Score Classification Dataset.csv')
#Age,Gender,Income,Education,Marital Status,Number of children,House ownership

In [ ]:
print(data.head())
print(data.info())
print(data.describe())

   Age  Gender  Income            Education Marital Status  \
0   25  Female   50000    Bachelor's Degree         Single   
1   30    Male  100000      Master's Degree        Married   
2   35  Female   75000            Doctorate        Married   
3   40    Male  125000  High School Diploma         Single   
4   45  Female  100000    Bachelor's Degree        Married   

   Number of Children Home Ownership Credit Score  
0                   0         Rented         High  
1                   2          Owned         High  
2                   1          Owned         High  
3                   0          Owned         High  
4                   3          Owned         High  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Age                 164 non-null    int64 
 1   Gender              164 non-null    object
 2   Income        

In [ ]:
# Handling categorical variables
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])
data['Education'] = label_encoder.fit_transform(data['Education'])
data['Marital Status'] = label_encoder.fit_transform(data['Marital Status'])
data['Home Ownership'] = label_encoder.fit_transform(data['Home Ownership'])

In [ ]:
print(data.isnull().sum())
# No missing values found in the provided data.


Age                   0
Gender                0
Income                0
Education             0
Marital Status        0
Number of Children    0
Home Ownership        0
Credit Score          0
dtype: int64


In [ ]:
# Step 2: Train-Test Split
X = data.drop('Credit Score', axis=1)
y = data['Credit Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 3: Model Building
# Initialize the model
model = Sequential()

# Add input layer with more neurons
model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))

# Add additional hidden layers with more neurons
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

# Add output layer
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize label encoder
label_encoder = LabelEncoder()

# Fit and transform the target variable
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
model.fit(X_train, y_train_encoded, epochs=200, batch_size=10, validation_data=(X_test, y_test_encoded))


Epoch 1/200
14/14 [==============================] - 1s 22ms/step - loss: 1239.4033 - accuracy: 0.6336 - val_loss: 31.5349 - val_accuracy: 0.6970
Epoch 2/200
14/14 [==============================] - 0s 7ms/step - loss: 138.6851 - accuracy: 0.6489 - val_loss: 18.3921 - val_accuracy: 0.1515
Epoch 3/200
14/14 [==============================] - 0s 5ms/step - loss: 73.4097 - accuracy: 0.6336 - val_loss: 2.8520 - val_accuracy: 0.6970
Epoch 4/200
14/14 [==============================] - 0s 5ms/step - loss: 169.7443 - accuracy: 0.6336 - val_loss: 2.4593 - val_accuracy: 0.6970
Epoch 5/200
14/14 [==============================] - 0s 5ms/step - loss: 112.4362 - accuracy: 0.6107 - val_loss: 174.7208 - val_accuracy: 0.1515
Epoch 6/200
14/14 [==============================] - 0s 6ms/step - loss: 179.3359 - accuracy: 0.6336 - val_loss: 8.1594 - val_accuracy: 0.6970
Epoch 7/200
14/14 [==============================] - 0s 5ms/step - loss: 85.2105 - accuracy: 0.6336 - val_loss: 79.6509 - val_accuracy: 0

In [ ]:
# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


2/2 [==============================] - 0s 9ms/step - loss: 0.2439 - accuracy: 0.6970
Test Loss: 0.24393530189990997
Test Accuracy: 0.6969696879386902


HPERPARAMETER TUNING

In [ ]:
#Applied Hyperparameter tuning

!pip install keras-tuner --upgrade
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l2
import kerastuner as kt
from tensorflow.keras.optimizers import Adam

# Load the dataset
# Assuming your data is already loaded in `df`
df = pd.read_csv('//content//Credit Score Classification Dataset.csv')

# Data preprocessing
# Encoding categorical variables
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Splitting dataset into features (x) and target label (y)
x = df.drop('Credit Score', axis=1)
y = df['Credit Score']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)

# Splitting data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Function to build the classification model
def build_classification_model(hp):
    model = Sequential()
    # Adding a dense layer with variable number of units and L2 regularization
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu',
                    input_dim=x_train.shape[1],
                    kernel_regularizer=l2(hp.Float('l2', 1e-4, 1e-2, sampling='log'))))
    # Adding dropout layer with variable dropout rate
    model.add(Dropout(rate=hp.Float('dropout', 0.2, 0.5, step=0.1)))

    # Adding another dense layer
    model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=512, step=32),
                    activation='relu',
                    kernel_regularizer=l2(hp.Float('l2_2', 1e-4, 1e-2, sampling='log'))))

    # Adding output layer for binary classification
    model.add(Dense(1, activation='sigmoid'))

    # Compiling the model with variable learning rate
    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


# Instantiate the tuner
tuner = kt.RandomSearch(
    build_classification_model,
    objective='val_accuracy',  # Use accuracy as the objective for classification
    max_trials=20,
    executions_per_trial=1,
    directory='my_dir',
    project_name='keras_classification_tuning')




Reloading Tuner from my_dir/keras_classification_tuning/tuner0.json


In [ ]:
# Perform hyperparameter search
tuner.search(x_train, y_train, epochs=500, validation_split=0.1)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the classification model with the optimal hyperparameters and train it
classification_model = build_classification_model(best_hps)
classification_model.fit(x_train, y_train, epochs=500, validation_split=0.1)



Epoch 1/500
4/4 [==============================] - 2s 166ms/step - loss: 0.7406 - accuracy: 0.4706 - val_loss: 0.6651 - val_accuracy: 0.6667
Epoch 2/500
4/4 [==============================] - 0s 20ms/step - loss: 0.7133 - accuracy: 0.6765 - val_loss: 0.6552 - val_accuracy: 0.8333
Epoch 3/500
4/4 [==============================] - 0s 20ms/step - loss: 0.6969 - accuracy: 0.7745 - val_loss: 0.6458 - val_accuracy: 0.8333
Epoch 4/500
4/4 [==============================] - 0s 31ms/step - loss: 0.6848 - accuracy: 0.6961 - val_loss: 0.6366 - val_accuracy: 0.9167
Epoch 5/500
4/4 [==============================] - 0s 26ms/step - loss: 0.6688 - accuracy: 0.8039 - val_loss: 0.6283 - val_accuracy: 1.0000
Epoch 6/500
4/4 [==============================] - 0s 30ms/step - loss: 0.6576 - accuracy: 0.8431 - val_loss: 0.6191 - val_accuracy: 1.0000
Epoch 7/500
4/4 [==============================] - 0s 26ms/step - loss: 0.6329 - accuracy: 0.8235 - val_loss: 0.6103 - val_accuracy: 1.0000
Epoch 8/500
4/4 [==

In [ ]:
# Evaluate the model on test set
test_loss, test_accuracy = classification_model.evaluate(x_test, y_test)
print(f'Test Accuracy with Hyperparameter Tuning: {test_accuracy}')

# Save the trained model
classification_model.save('classification_model.h5')

2/2 [==============================] - 0s 12ms/step - loss: -7.9720 - accuracy: 0.7600
Test Accuracy with Hyperparameter Tuning: 0.7599999904632568


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Stacked ensembling

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
# Assuming your data is already loaded in `df`

# Data preprocessing
# Encoding categorical variables
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Splitting dataset into features (x) and target label (y)
x = df.drop('Credit Score', axis=1)
y = df['Credit Score']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)

# Splitting data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the base models
# You may need to adjust hyperparameters or use different algorithms based on your data characteristics
nn_model = Sequential([
    Dense(256, input_dim=x_train.shape[1], activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')  # Adjust units to match the number of classes
])
nn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
nn_model.fit(x_train, to_categorical(y_train), epochs=100, batch_size=10, validation_split=0.1)

rf_model = RandomForestClassifier().fit(x_train, y_train)
gb_model = GradientBoostingClassifier().fit(x_train, y_train)

# Predict on the hold-out set to create meta-features
nn_test_predictions = nn_model.predict(x_test)
rf_test_predictions = rf_model.predict(x_test)
gb_test_predictions = gb_model.predict(x_test)

# Stack test predictions to form meta-features for final prediction
meta_test_features = np.column_stack((nn_test_predictions, rf_test_predictions, gb_test_predictions))

# Train the meta-model
meta_model = Sequential([
    Dense(64, input_dim=meta_test_features.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')  # Adjust units to match the number of classes
])
meta_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
meta_model.fit(meta_test_features, to_categorical(y_test), epochs=100, batch_size=10, validation_split=0.1)

# Final prediction and evaluation
# You may need to adjust this part based on the specifics of your problem
final_test_predictions = meta_model.predict(meta_test_features)
accuracy = accuracy_score(y_test, np.argmax(final_test_predictions, axis=1))
print(f'Accuracy of Stacked Ensemble: {accuracy}')


Epoch 1/100
12/12 [==============================] - 2s 22ms/step - loss: 0.9612 - accuracy: 0.7265 - val_loss: 0.7395 - val_accuracy: 0.9286
Epoch 2/100
12/12 [==============================] - 0s 7ms/step - loss: 0.5157 - accuracy: 0.8974 - val_loss: 0.2465 - val_accuracy: 0.9286
Epoch 3/100
12/12 [==============================] - 0s 7ms/step - loss: 0.2535 - accuracy: 0.8974 - val_loss: 0.1828 - val_accuracy: 0.9286
Epoch 4/100
12/12 [==============================] - 0s 5ms/step - loss: 0.2156 - accuracy: 0.8974 - val_loss: 0.2337 - val_accuracy: 0.9286
Epoch 5/100
12/12 [==============================] - 0s 6ms/step - loss: 0.1969 - accuracy: 0.9402 - val_loss: 0.2761 - val_accuracy: 0.9286
Epoch 6/100
12/12 [==============================] - 0s 8ms/step - loss: 0.1762 - accuracy: 0.9145 - val_loss: 0.3438 - val_accuracy: 0.9286
Epoch 7/100
12/12 [==============================] - 0s 7ms/step - loss: 0.1641 - accuracy: 0.9487 - val_loss: 0.3191 - val_accuracy: 0.9286
Epoch 8/100


2/2 [==============================] - 0s 4ms/step
Accuracy of Stacked Ensemble: 1.0


In [ ]:
print(f'Accuracy of Stacked Ensemble: {accuracy}')


Accuracy of Stacked Ensemble: 1.0


Confusion Matrix



In [ ]:
#making confusion matrix
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Convert final_test_predictions to class labels
final_test_predictions_labels = np.argmax(final_test_predictions, axis=1)

# Calculate precision
precision = precision_score(y_test, final_test_predictions_labels, average='weighted')
print(f'Precision: {precision}')

# Calculate recall
recall = recall_score(y_test, final_test_predictions_labels, average='weighted')
print(f'Recall: {recall}')

# Calculate F1-score
f1 = f1_score(y_test, final_test_predictions_labels, average='weighted')
print(f'F1-score: {f1}')

# Create confusion matrix
conf_matrix = confusion_matrix(y_test, final_test_predictions_labels)
print('Confusion Matrix:')
print(conf_matrix)


Precision: 1.0
Recall: 1.0
F1-score: 1.0
Confusion Matrix:
[[ 5  0  0]
 [ 0 23  0]
 [ 0  0  5]]


Doing Cross validation

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score

# Load the dataset (assuming it's already loaded in `df`)
# Your data preprocessing steps (label encoding, feature scaling) here...

# Define the number of folds for cross-validation
num_folds = 5

# Initialize StratifiedKFold
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Lists to store the accuracy scores for each fold
accuracy_scores = []

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(X_scaled, y)):
    print(f'Fold {fold + 1}:')
    X_train, X_val = X_scaled[train_index], X_scaled[test_index]
    y_train, y_val = y[train_index], y[test_index]

    # Define and train your model
    model = Sequential([
        Dense(256, input_dim=X_train.shape[1], activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(len(np.unique(y)), activation='softmax')  # Adjust units to match the number of classes
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, to_categorical(y_train), epochs=100, batch_size=10, validation_split=0.1, verbose=0)

   # Evaluate the model on the validation set
    val_probabilities = model.predict(X_val)
    val_predictions = np.argmax(val_probabilities, axis=1)
    accuracy = accuracy_score(y_val, val_predictions)
    print(f'Validation Accuracy: {accuracy}')
    accuracy_scores.append(accuracy)

# Calculate and print the mean accuracy across all folds
mean_accuracy = np.mean(accuracy_scores)
print(f'Mean Accuracy across {num_folds} folds: {mean_accuracy}')


Fold 1:


2/2 [==============================] - 0s 8ms/step
Validation Accuracy: 1.0
Fold 2:
2/2 [==============================] - 0s 8ms/step
Validation Accuracy: 0.9696969696969697
Fold 3:
2/2 [==============================] - 0s 9ms/step
Validation Accuracy: 1.0
Fold 4:
2/2 [==============================] - 0s 5ms/step
Validation Accuracy: 0.9696969696969697
Fold 5:
1/1 [==============================] - 0s 74ms/step
Validation Accuracy: 0.96875
Mean Accuracy across 5 folds: 0.9816287878787879


L2 Regularization

In [ ]:
from keras.layers import Dropout
from keras import regularizers

# Define the base models with regularization
nn_model = Sequential([
    Dense(512, input_dim=x_train.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(len(np.unique(y)), activation='softmax')  # Adjust units to match the number of classes
])

# Compile the model
nn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with early stopping
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

history = nn_model.fit(x_train, to_categorical(y_train), epochs=500, batch_size=10, validation_split=0.1, callbacks=[early_stopping])


Epoch 1/500
11/11 [==============================] - 2s 32ms/step - loss: 7.2354 - accuracy: 0.4216 - val_loss: 6.6266 - val_accuracy: 0.7500
Epoch 2/500
11/11 [==============================] - 0s 11ms/step - loss: 6.3666 - accuracy: 0.5588 - val_loss: 5.8798 - val_accuracy: 0.7500
Epoch 3/500
11/11 [==============================] - 0s 9ms/step - loss: 5.6773 - accuracy: 0.7157 - val_loss: 5.1480 - val_accuracy: 0.7500
Epoch 4/500
11/11 [==============================] - 0s 11ms/step - loss: 5.0629 - accuracy: 0.6961 - val_loss: 4.4957 - val_accuracy: 1.0000
Epoch 5/500
11/11 [==============================] - 0s 10ms/step - loss: 4.4925 - accuracy: 0.7059 - val_loss: 4.0042 - val_accuracy: 1.0000
Epoch 6/500
11/11 [==============================] - 0s 10ms/step - loss: 4.0390 - accuracy: 0.7549 - val_loss: 3.6226 - val_accuracy: 1.0000
Epoch 7/500
11/11 [==============================] - 0s 14ms/step - loss: 3.6798 - accuracy: 0.7647 - val_loss: 3.2922 - val_accuracy: 1.0000
Epoch 8

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = nn_model.evaluate(x_test, to_categorical(y_test))

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


2/2 [==============================] - 0s 8ms/step - loss: 0.6046 - accuracy: 0.8200
Test Loss: 0.6046128273010254
Test Accuracy: 0.8199999928474426
